Make sure all required directories are here

In [33]:
import os

original_path = './../data/1_original'
intermediate_path = './../data/2_intermediate'

def assert_directory_exists(dir):
    if not os.path.exists(dir):
        assert_directory_exists(os.path.pardir)
        os.makedirs(dir)

assert_directory_exists(original_path)
assert_directory_exists(intermediate_path)
assert_directory_exists('./../data/3_prepared')
assert_directory_exists('./../data/4_result')


Download all files

In [34]:
import wget

files = {
    'https://data.cityofnewyork.us/api/views/bkjd-kr4k/rows.csv?accessType=DOWNLOAD':'brooklyn_schools.csv',
    'https://www1.nyc.gov/assets/finance/downloads/pdf/rolling_sales/rollingsales_brooklyn.xls': 'rollingsales_brooklyn.xls',
    'https://data.cityofnewyork.us/api/views/uip8-fykc/rows.csv?accessType=DOWNLOAD':'NYPD_Arrest_Data_Year_to_Date.csv',
    'https://data.cityofnewyork.us/api/views/5ucz-vwe8/rows.csv?accessType=DOWNLOAD': 'NYPD_Shooting_Incident_Data_Year_To_Date.csv',
    'https://data.cityofnewyork.us/api/geospatial/tqmj-j8zm?method=export&format=Shapefile':'Borough_Boundaries.zip',
    'https://data.cityofnewyork.us/download/i8iw-xf4u/application%2Fzip':'ZIP_codes.zip'
     }

for src, dest_name in files.items():
    dest_path = os.path.join(original_path, dest_name)
    if os.path.exists(dest_path):
         print(f"'{dest_name}' is present")    
    else:
        print(f"Downloading '{dest_name}'")    
        wget.download(src, dest_path)

print("done")


'brooklyn_schools.csv' is present
'rollingsales_brooklyn.xls' is present
'NYPD_Arrest_Data_Year_to_Date.csv' is present
'NYPD_Shooting_Incident_Data_Year_To_Date.csv' is present
done


Unzip all zip archives and delete the archive

In [35]:
import zipfile

def unzip(file):
    unziped_dir = os.path.join(original_path, file.name[0: -4])        
    if not os.path.exists(unziped_dir):
        print(f"unzipping '{file.name}'")
        with zipfile.ZipFile(file.path, 'r') as zip_ref:
            zip_ref.extractall(unziped_dir)

for entry in os.scandir(original_path):
    if entry.is_file() and entry.name.endswith('.zip'):
        unzip(entry)
        os.remove(entry)
print("done")
        

done


Convert the shape files to geo-jsons

In [36]:
import gdal

# copied from https://shallowsky.com/blog/mapping/folium-with-shapefiles.html
def shapefile2geojson(infile, outfile, fieldname='geometry'):
    '''Translate a shapefile to GEOJSON.'''
    options = gdal.VectorTranslateOptions(format="GeoJSON",
                                          dstSRS="EPSG:4326")
    gdal.VectorTranslate(outfile, infile, options=options)

def find_shape_file(search_path):
    for entry in os.scandir(search_path):
        if entry.is_file() and entry.name.endswith('.shp'):
            return entry.path
    return None

for entry in os.scandir(original_path):
    if entry.is_dir():
        shp_file_path = find_shape_file(entry.path)
        if shp_file_path is not None:
            dest_file_path = os.path.join(intermediate_path, entry.name)+".geojson"
            if not os.path.exists(dest_file_path):
                print(f"converting '{entry.name}''")
                shapefile2geojson(shp_file_path, dest_file_path)

print("done")

done


Create a geo-json file with the border of Brooklyn

In [37]:
import json
from shapely.geometry.polygon import Polygon

def get_largest_polygon(polygons):
    """ Pick the polygon with the largest area.
    ATTENTION: do not pass None or empty polygons in!
    ATTENTION: This is calculated on a plane, coordinates are not consideret to be polar coordinates!
    For the case at hand this works fine because the most of brooklyn is one large polygon and all others are small islands"""
    i = 0
    index_of_largest = -1
    area_of_largest = -1
    for bds in polygons:
        area = Polygon(bds[0]).area
        if area > area_of_largest:
            index_of_largest = i
            area_of_largest = area
        i = i+1

    return polygons[index_of_largest]


def write_geojson(polygon, name, file_path):
    """ Writes a geojson file with one feature, a polygon"""
    geom = {'type': 'Polygon', 'coordinates':polygon}
    result = {
                'type': 'Feature',
                'properties':{'name': name},
                'geometry': geom
            }

    with open(file_path, 'w') as fp:
                json.dump(result, fp)

dest = './../data/2_intermediate/brooklyn_border.geojson'

if (os.path.exists(dest)):
    print("brooklyn_border.geojson exists")
else:
    print("extracting brooklyn_border.geojson")
    with open('./../data/2_intermediate/Borough_Boundaries.geojson') as file:
        data = json.load(file)
        for feature in data['features']:

            # pick Brooklyn
            if feature['properties']['boro_name'] == 'Brooklyn':

                # pick the largest polygon
                largest = get_largest_polygon(feature['geometry']['coordinates'])
                write_geojson(largest, "largest polygon of Brooklyn", dest)

    print("done")


brooklyn_border.geojson exists


Bonus cell: show all polygons from Brooklyn, the 'main' one green and the other red

In [42]:
# write all_brooklyn_borders.geojson
with open('./../data/2_intermediate/Borough_Boundaries.geojson') as file:
    data = json.load(file)
    for feature in data['features']:
        if feature['properties']['boro_name'] == 'Brooklyn':
            new_features = []

            i = 0
            for coordinate in feature['geometry']['coordinates']:
                geom = {'type': 'Polygon', 'coordinates': coordinate}
                new_feature = {'type': 'Feature', 'properties':{'number':i}, 'geometry':geom}
                
                new_features.append(new_feature)          
                i = i+1
            new = {
                'type': 'FeatureCollection',
                'features':new_features
                }

            
            with open('./../data/2_intermediate/brooklyn_borders_all.geojson', 'w') as fp:
                json.dump(new, fp)


import folium

m = folium.Map(
    location=(40.64558018290456, -73.94975979616603), 
    zoom_start=12,
    tiles='Stamen Toner'
) 

def style_function(x):
    number = x['properties']['number']
    return {
        'color': '#339933' if number==27 else '#CC0000',
        'opacity': 0.5,
        'weigth': 1,
        'fillOpacity': 0.5,
        'fill':'#00FF00'
    }

folium.GeoJson(
    "./../data/2_intermediate/brooklyn_borders_all.geojson",
    name = "All Brooklyn borders",

    tooltip=folium.features.GeoJsonTooltip(fields=['number'], aliases=['#']),
    style_function=style_function
).add_to(m) 

m           